## Layer-wise Relevance Propagation

Implement LRP using iNNvestigate package: https://innvestigate.readthedocs.io/en/latest/index.html. iNNvestigate currently works with keras models and tensorflow 1.X. In this notebook, the sequential model is recreated in keras, and the trained weights are loaded. 

link to older Keras documentation: https://faroit.com/keras-docs/2.0.2/metrics/#binary_accuracy

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
import keras

Using TensorFlow backend.


In [3]:
import innvestigate

In [4]:
import importlib

In [5]:
## project_utils are functions/variables defined by me
from project_utils import parameters as param
from project_utils import utils as util
from project_utils import read_utils as read

importlib.reload(read)
importlib.reload(util)
importlib.reload(param)

<module 'project_utils.parameters' from '/oak/stanford/groups/omramom/group_members/fvdav/projects/precip_clustering/project_utils/parameters.py'>

### Build keras model and load pre-trained weights

In [6]:
## slightly different syntax to build model in TF1 (comments note differences)
def build_model_keras(lr = .0004, conv_filters = 16, dense_neurons = 16, 
                             dense_layers = 1, 
                activity_reg = 0.001, dropout_rate = 0.2, input_channels = 2):

    model = keras.models.Sequential() 
    model.add(keras.layers.InputLayer(input_shape=(15, 35, input_channels))) ## define input shape
    ## InputLayer instead of Input, input_shape instead of shape
    
    model.add(keras.layers.Conv2D(conv_filters, 
                            (3,3), activity_regularizer=keras.regularizers.l2(0.01))) 
    model.add(keras.layers.Activation('relu')) 
    model.add(keras.layers.MaxPooling2D((2,2)))
                            
    model.add(keras.layers.Conv2D(conv_filters, (3,3), activity_regularizer=keras.regularizers.l2(activity_reg))) 
    model.add(keras.layers.Activation('relu')) 
    model.add(keras.layers.MaxPooling2D((2,2))) 
                            
    model.add(keras.layers.Flatten()) 
    for i in range(dense_layers):
        model.add(keras.layers.Dense(dense_neurons, activity_regularizer=keras.regularizers.l2(activity_reg)))
        model.add(keras.layers.Activation('relu'))
     
    model.add(keras.layers.Dense(2, activation='softmax')) 
    
    model.compile(loss=keras.losses.categorical_crossentropy, ## instead of CategoricalCrossentropy
                  optimizer=keras.optimizers.Adam(lr = lr), ## lr instead of learning_rate
                  metrics=['categorical_accuracy'])
    return(model)

In [7]:
model = build_model_keras()

In [8]:
model.load_weights("../processed_data/trained_weights.h5")

In [9]:
x_dat = read.get_hgt_slp_input()
y_dat = util.get_precip_classes(pd.read_csv("../processed_data/region_mean_precip.csv")["prcp"], 
                               q = [0.95])

In [10]:
y_dat_onehot = util.onehot(y_dat)

In [11]:
ind = np.arange(len(y_dat))

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test, ind_train, ind_test = train_test_split(x_dat, y_dat, ind, test_size=0.25, random_state=42, 
                                                   shuffle = True, stratify = y_dat)

#### compare predictions to TF2

In [14]:
model.evaluate(x_dat, y_dat_onehot)

14244/14244 [==============================] - 2s 115us/step


[4.879402918264592, 0.8803706823925863]

^ same accuracy as tf2 model (very small error <0.0000001)

In [15]:
predict_df = pd.read_csv("../processed_data/predicted_class_data.csv")

In [16]:
test_predictions = predict_df.loc[predict_df.set == 'test']

In [17]:
test_predictions_tf1 = np.argmax(model.predict(x_dat)[np.sort(ind_test)], axis = 1)

In [18]:
np.sum(test_predictions_tf1 != test_predictions.predicted_class)

0

^ all predictions match. (Also checked that probabilities match)

## LRP

see Montavon et al. (https://www.sciencedirect.com/science/article/pii/S1051200417302385)

LRP algorithm using Alpha-Beta rule - Alpha1Beta0 only tracks positive relevance 

In [19]:
model_strip = innvestigate.utils.model_wo_softmax(model)

In [20]:
## create analyzer object 
lrp_analyzerA1B0 = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha1Beta0(model_strip)

### save relevance data for all days

In [33]:
rel_all = lrp_analyzerA1B0.analyze(x_dat)

In [34]:
rel_data = xr.Dataset(
    data_vars=dict(
        rel_hgt=(["time", "lat", "lon"], rel_all[:,:,:,0]), 
        rel_slp=(["time", "lat", "lon"], rel_all[:,:,:,1]),
    ),
    coords=dict(
        time=predict_df.date.values, 
        lat = param.lats, 
        lon = param.lons),
)

In [35]:
rel_data

<xarray.Dataset>
Dimensions:  (lat: 15, lon: 35, time: 14244)
Coordinates:
  * time     (time) object '1981-01-01' '1981-01-02' ... '2019-12-31'
  * lat      (lat) float32 55.0 52.5 50.0 47.5 45.0 ... 30.0 27.5 25.0 22.5 20.0
  * lon      (lon) float32 220.0 222.5 225.0 227.5 ... 297.5 300.0 302.5 305.0
Data variables:
    rel_hgt  (time, lat, lon) float32 5.161e-05 7.148e-05 3.191e-05 ... 0.0 -0.0
    rel_slp  (time, lat, lon) float32 0.0 1.016e-05 5.299e-05 ... -0.0 -0.0 -0.0

In [36]:
rel_data.to_netcdf("../processed_data/lrp_data_all_days.nc")